In [ ]:
from flask import Flask, render_template, request, redirect, url_for, jsonify
from pymongo import MongoClient
from flask_cors import CORS, cross_origin
import json
from bson.objectid import ObjectId

#creating flask instance
app = Flask(__name__)
CORS(app, support_credentials = True)

#creating a connection with the mongodb server and setting up the db and collection which are to be used
client = MongoClient()
db = client['contacts_db']
collection = db['contacts']

#home page with all the contacts in the list(collection)
@app.route('/')
@cross_origin(supports_credentials = True)
def index():
    contacts = list(collection.find())
    contacts = [{'_id': str(contact['_id']), 'name': contact['name'], 'email': contact['email'], 'phone': contact['phone']} for contact in contacts]
    return render_template('index.html', contacts=contacts)

@app.route('/add')
def add():
    return render_template('add.html')

#adding a contact into the collection
@app.route('/adddoc', methods=['POST'])
@cross_origin(supports_credentials = True)
def add_contact():
        data = request.get_json()
        contact = {
            'name': data['name'],
            'email': data['email'],
            'phone': data['phone']
        }
        collection.insert_one(contact)
        return json.dumps({'success': True}), 200, {'ContentType': 'application/json'}

#editing a contact according to the user's needs
@app.route('/edit/<id>', methods=['GET', 'POST'])
@cross_origin(supports_credentials = True)
def edit(id):
    contact = collection.find_one({'_id': ObjectId(id)})
    if request.method == 'POST':
        update_data = request.get_json()
        contact = {
            'name': data['name'],
            'email': data['email'],
            'phone': data['phone']
        }
        collection.update_one({'_id': ObjectId(id)}, {'$set': contact})
        return json.dumps({'success': True}), 200, {'ContentType': 'application/json'}

#deleting a contact from the contact list
@app.route('/delete/<id>', methods=['POST'])
@cross_origin(supports_credentials = True)
def delete(id):
    collection.delete_one({'_id': ObjectId(id)})
    return redirect(url_for('index'))

app.run(host = "localhost",port=8080)